In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

options = Options()
options.headless = False
# options.add_argument('window-size=1920x1080')

web = 'https://in.sugarcosmetics.com/lips/lipsticks/transferproof-lipsticks'
driver = webdriver.Chrome(options=options)
driver.get(web)
driver.maximize_window()

# Use WebDriverWait to wait for the container to be present
container = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'DesktopProductCard_collectionContainer__j0VId'))
)

# Get initial product count
product_count = len(
    driver.find_elements(By.XPATH, '//div[contains(@class,"DesktopProductCard_collectionCard__pSeq7")]'))

# Set a delay for scrolling (adjust as needed)
scroll_delay = 2

# Scroll until no more new products are loaded
while True:
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for new products to load
    time.sleep(scroll_delay)

    # Get the updated product count
    new_product_count = len(
        driver.find_elements(By.XPATH, '//div[contains(@class,"DesktopProductCard_collectionCard__pSeq7")]'))

    # Check if new products are loaded
    if new_product_count == product_count:
        break

    # Update the product count
    product_count = new_product_count

# Find elements within the container after scrolling
products = container.find_elements(By.XPATH, './/div[contains(@class,"DesktopProductCard_collectionCard__pSeq7")]')

product_data = []

try:
    for product in products:
        product_link = product.find_element(By.XPATH, './/a[contains(@class,"w-100")]').get_attribute("href")
        product_name = product.find_element(By.XPATH, './/p[contains(@class,"truncate-line-clamp_2 pt-2 px-5")]').text
        product_price = product.find_element(By.XPATH,
                                             './/div[contains(@class,"DesktopProductCard_pricelayer21__n_d5C")]').text

        product_data.append(
            {'Product Link': product_link, 'Product Name': product_name, 'Product Price': product_price})
except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()

# Create a DataFrame from the scraped data
df_products = pd.DataFrame(product_data)

# Save the DataFrame to an Excel file
df_products.to_excel('cosmetics_data.xlsx', index=False)
